In [119]:
!pip install gradio openai-whisper transformers fpdf torch

In [115]:
import gradio as gr
import whisper
from transformers import pipeline
from fpdf import FPDF
import os









In [116]:
# Load Whisper ASR Model
model = whisper.load_model("base")
# Load Summarization Model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")



/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
Device set to

In [109]:
def transcribe_audio(audio_file):
    """Convert speech to text using Whisper"""

    if not os.path.exists(audio_file):
        return "Error: Audio file not found!"

    try:
        result = model.transcribe(audio_file)
        return result["text"]
    except Exception as e:
        return f"Transcription failed: {str(e)}"

Device set to use cpu


In [110]:
def summarize_text(text):
    """Summarizes long transcriptions into a short summary"""

    if len(text) < 50:
        return "Text is too short for summarization!"

    try:
        summary = summarizer(text, max_length=100, min_length=30, do_sample=False)
        return summary[0]["summary_text"]
    except Exception as e:
        return f"Summarization failed: {str(e)}"



In [117]:
def save_as_pdf(transcription):
    """Save the transcribed text as a PDF file"""

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, transcription)

    pdf_path = "/content/transcription.pdf"
    pdf.output(pdf_path)
    return pdf_path


In [120]:
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🎤 Speech-to-Text & Summarization App")
    gr.Markdown("Convert audio into text and get a summary instantly!")

    # Upload Audio Section
    audio_input = gr.File(label="🎵 Upload Your Audio File (MP3, WAV, M4A)")

    # Buttons for Actions
    transcribe_btn = gr.Button("📝 Transcribe")
    summarize_btn = gr.Button("📑 Summarize Text")
    download_btn = gr.Button("📥 Download as PDF")

    # Output Sections
    transcription_output = gr.Textbox(label="Transcription", placeholder="Your transcription will appear here...", lines=10)
    summary_output = gr.Textbox(label="Summary", placeholder="Summary of your text...", lines=5)
    pdf_output = gr.File(label="Download PDF")

    # Button Actions
    transcribe_btn.click(transcribe_audio, inputs=audio_input, outputs=transcription_output)
    summarize_btn.click(summarize_text, inputs=transcription_output, outputs=summary_output)
    download_btn.click(save_as_pdf, inputs=transcription_output, outputs=pdf_output)

# Launch the app
app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://427349263d49a79d81.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
